# Building a Exa (formerly Metaphor) Data Agent

<a href="https://colab.research.google.com/github/run-llama/llama-hub/blob/main/llama_hub/tools/notebooks/exa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This tutorial walks through using the LLM tools provided by the [Exa API](https://exa.ai) to allow LLMs to easily search and retrieve HTML content from the Internet.

To get started, you will need an [OpenAI api key](https://platform.openai.com/account/api-keys) and an [Exa API key](https://dashboard.exa.ai/overview)

We will import the relevant agents and tools and pass them our keys here:

In [ ]:
!pip install exa_py

In [ ]:
# Set up OpenAI
import os
import openai
from llama_index.agent import OpenAIAgent

openai.api_key = os.environ["OPENAI_API_KEY"]

# Set up Metaphor tool
from llama_hub.tools.exa.base import ExaToolSpec

exa_tool = ExaToolSpec(
    api_key=os.environ["EXA_API_KEY"],
    # max_characters=2000   # this is the default
)

exa_tool_list = exa_tool.to_tool_list()
for tool in exa_tool_list:
    print(tool.metadata.name)

search
retrieve_documents
search_and_retrieve_documents
search_and_retrieve_highlights
find_similar
current_date


## Testing the Exa tools

We've imported our OpenAI agent, set up the api key, and initialized our tool, checking the methods that it has available. Let's test out the tool before setting up our Agent.

All of the Exa search tools make use of the `AutoPrompt` option where Exa will pass the query through an LLM to refine and improve it.

In [ ]:
exa_tool.search_and_retrieve_documents("machine learning transformers", num_results=3)

[Exa Tool] Autoprompt: Here is a link to an article about machine learning transformers:


[Document(id_='a2eef546-811c-4bbf-bfe0-1c7fd7fe526d', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='b846333f5fffd9a45abfcb16388bb7192676eee95333ff623321c092c24d5bb2', text='\n \n \n \n \n \n \n \n \n \n \n December 18, 2021\n \n \n \n \n \n \n \n \n \n 7 minute read\n \n \n \n \n \n \n \n \n \nSummary\nTransformers architectures are the hottest thing in supervised and unsupervised learning, achieving SOTA results on natural language processing, vision, audio and multimodal tasks. Their key capability is to capture which elements in a long sequence are worthy of attention, resulting in great summarisation and generative skills. Can we transfer any of these skills to reinforcement learning? The ans', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='174bd84d-6913-43cd-b193-b807bfd1f6b8', embedding=None, meta

In [ ]:
exa_tool.find_similar(
    "https://www.mihaileric.com/posts/transformers-attention-in-disguise/"
)

[{'title': 'A Deep Dive Into the Transformer Architecture — The Development of Transformer Models',
  'url': 'https://towardsdatascience.com/a-deep-dive-into-the-transformer-architecture-the-development-of-transformer-models-acbdf7ca34e0?gi=b4d77d2ab4db',
  'id': '60J3eIu_oZO9OEulMglxuw'},
 {'title': 'What is a Transformer?',
  'url': 'https://medium.com/inside-machine-learning/what-is-a-transformer-d07dd1fbec04',
  'id': 'uxGX5rLD8HXrmgQiyQIYyw'},
 {'title': 'The Transformer Model',
  'url': 'https://towardsdatascience.com/attention-is-all-you-need-e498378552f9?gi=92758857966b',
  'id': 'RKL4_dd9kKX_OThZCXo8Yg'}]

In [ ]:
exa_tool.search_and_retrieve_documents(
    "This is a summary of recent research around diffusion models:", num_results=1
)

[Exa Tool] Autoprompt: Here is a recent research paper about diffusion models:


[Document(id_='556c9b58-67d1-470a-b03a-1596d1d6c7dd', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Diffusion Models Beat GANs on Image Synthesis\n\nDate Published: 2021-06-01\n\nAuthors:\nPrafulla  Dhariwal, prafulla@openai.com\nAlex  Nichol\n\nAbstract\n\nWe show that diffusion models can achieve image sample quality superior to the current state-of-the-art generative models. We achieve this on unconditional image synthesis by finding a better architecture through a series of ablations. For conditional image synthesis, we further improve sample quality with classifier guidance: a simple, compute-efficient method for trading off diversity for fidelity using gradients from a classifier. We achieve an FID of 2.97 on ImageNet 128×128, 4.59 on ImageNet 256×256, and 7.72 on ImageNet 512×512, and we match BigGAN-deep even with as few as 25 forward passes per sample, all while maintaining better coverage of the distributi

While `search_and_retrieve_documents` returns raw text from the source document, `search_and_retrieve_highlights` returns relevant curated snippets.

In [ ]:
exa_tool.search_and_retrieve_highlights(
    "This is a summary of recent research around diffusion models:", num_results=1
)

[Exa Tool] Autoprompt: Here is a research paper about diffusion models that you might find useful:


[Document(id_='695ad95b-edd8-4083-ad16-512203d43f9a', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='On a high level, diffusion models sample from a distribution by reversing a gradual noising process. In particular, sampling starts with noise x T and produces gradually less-noisy samples x T -1 , x T -2 , ... until reaching a final sample x 0 . Each timestep t corresponds to a certain noise level, and x t can be thought of as a mixture of a signal x 0 with some noise where the signal to noise ratio is determined by the timestep t. For the remainder of this paper, we assume that the noise is drawn from a diagonal Gaussian distribution, which works well for natural images and simplifies various derivations. A diffusion model learns to produce a slightly more "denoised" x t-1 from x t .', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_se

We can see we have different tools to search for results, retrieve the results, find similar results to a web page, and finally a tool that combines search and document retrieval into a single tool. We will test them out in LLM Agents below:

### Using the Search and Retrieve documents tools in an Agent

We can create an agent with access to the above tools and start testing it out:

In [ ]:
# We don't give the Agent our unwrapped retrieve document tools, instead passing the wrapped tools
agent = OpenAIAgent.from_tools(
    exa_tool_list,
    verbose=True,
)

In [ ]:
print(agent.chat("What are the best resturants in toronto?"))

Added user message to memory: What are the best resturants in toronto?
=== Calling Function ===
Calling function: search with args: {
  "query": "best restaurants in Toronto"
}
[Exa Tool] Autoprompt: Here's a great restaurant to try in Toronto:
Got output: [{'title': 'PATOIS • TORONTO', 'url': 'https://www.patoistoronto.com/', 'id': '5EC2l7fbaPoEydNVNwjc-A'}, {'title': 'Location', 'url': 'https://osteriagiulia.ca/', 'id': 'mpjelsyCOpNipFFI5AoZTQ'}, {'title': 'Enigma Yorkville | Modern European Restaurant in Toronto, ON', 'url': 'https://www.enigmayorkville.com/', 'id': 'jBOC2QfhTfuPjt0YdibEVA'}, {'title': 'GOA | by Hemant Bhagwani', 'url': 'https://www.goatoronto.ca/', 'id': 'e8sCvTX5NVVbwzoOr0o6aw'}, {'title': 'Portuguese inspired seafood from around the world | Adega Restaurante', 'url': 'https://adegarestaurante.ca/', 'id': 'oQiAWWgzrU-ryPNmgj3UuA'}, {'title': 'Home', 'url': 'https://www.avelorestaurant.com/', 'id': 'NDfST6oMKpJ0I-VYUf_WHA'}, {'title': 'PAI Northern Thai Kitchen', '

In [ ]:
print(agent.chat("tell me more about Osteria Giulia"))

Added user message to memory: tell me more about Osteria Giulia
=== Calling Function ===
Calling function: retrieve_documents with args: {
  "ids": ["mpjelsyCOpNipFFI5AoZTQ"]
}
Got output: Error: 'results'

=== Calling Function ===
Calling function: search with args: {
  "query": "Osteria Giulia Toronto"
}
[Exa Tool] Autoprompt: Here's a great Italian restaurant in Toronto called Osteria Giulia:
Got output: [{'title': 'Giulietta', 'url': 'https://giu.ca/', 'id': 'GWjEUcgDP26STPskrgMM5g'}, {'title': 'Location', 'url': 'https://osteriagiulia.ca/', 'id': 'mpjelsyCOpNipFFI5AoZTQ'}, {'title': 'Gusto 501', 'url': 'https://www.restaurantji.com/on/toronto/gusto-501-/', 'id': 'Oa5NvNXMnweapgE1tO8WYQ'}, {'title': 'Menu - Gia Restaurant', 'url': 'https://giarestaurant.ca/', 'id': '35CymOCvS59-Lmdilq5XLw'}, {'title': 'Osteria Ilaria', 'url': 'https://www.osteriailaria.com/', 'id': 'SN6NKvpXluVgWCv5GPI88Q'}, {'title': 'Osteria Fortunato', 'url': 'https://osteriafortunato.ca/', 'id': 'MC6pGvtQMnb6Iu

## Avoiding Context Window Issues

The above example shows the core uses of the Exa tool. We can easily retrieve a clean list of links related to a query, and then we can fetch the content of the article as a cleaned up html extract. Alternatively, the search_and_retrieve_documents tool directly returns the documents from our search result.

We can see that the content of the articles is somewhat long and may overflow current LLM context windows.  

1. Use `search_and_retrieve_highlights`: This is an endpoint offered by Exa that directly retrieves relevant highlight snippets from the web, instead of full web articles. As a result you don't need to worry about indexing/chunking offline yourself!

2. Wrap `search_and_retrieve_documents` with `LoadAndSearchToolSpec`: We set up and use a "wrapper" tool from LlamaIndex that allows us to load text from any tool into a VectorStore, and query it for retrieval. This is where the `search_and_retrieve_documents` tool become particularly useful. The Agent can make a single query to retrieve a large number of documents, using a very small number of tokens, and then make queries to retrieve specific information from the documents.

### 1. Using `search_and_retrieve_highlights`

The easiest is to just use `search_and_retrieve_highlights` from Exa. This is essentially a "web RAG" endpoint - they handle chunking/embedding under the hood.

In [ ]:
tools = exa_tool.to_tool_list(
    spec_functions=["search_and_retrieve_highlights", "current_date"]
)

In [ ]:
agent = OpenAIAgent.from_tools(
    tools,
    verbose=True,
)

In [ ]:
response = agent.chat("Tell me more about the recent news on semiconductors")
print(f"Response: {str(response)}")

Added user message to memory: Tell me more about the recent news on semiconductors
=== Calling Function ===
Calling function: current_date with args: {}
Got output: 2024-01-25

=== Calling Function ===
Calling function: search_and_retrieve_highlights with args: {
  "query": "semiconductors",
  "num_results": 5,
  "start_published_date": "2024-01-01",
  "end_published_date": "2024-01-25"
}
[Exa Tool] Autoprompt: "Here is an informative resource about semiconductors:
Got output: [Document(id_='e72b143e-df3a-408e-b790-24a3931b18da', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='<p>Semiconductors are crucial for multiple applications in electronics, communications, automotive, agriculture, healthcare, finance, and energy. After a slight turmoil phase during 2023, it is expected that the chip industry will exhibit a strong recovery and thrive this year and beyond.</p><p>According to the latest forecast by the World Semic

### 2. Using `LoadAndSearchToolSpec`

Here we wrap the `search_and_retrieve_documents` functionality with the `load_and_search_tool_spec`.

In [ ]:
from llama_index.tools.tool_spec.load_and_search.base import LoadAndSearchToolSpec

# The search_and_retrieve_documents tool is the third in the tool list, as seen above
search_and_retrieve_docs_tool = exa_tool.to_tool_list(
    spec_functions=["search_and_retrieve_documents"]
)[0]
date_tool = exa_tool.to_tool_list(spec_functions=["current_date"])[0]
wrapped_retrieve = LoadAndSearchToolSpec.from_defaults(search_and_retrieve_docs_tool)

Our wrapped retrieval tools separate loading and reading into separate interfaces. We use `load` to load the documents into the vector store, and `read` to query the vector store. Let's try it out again

In [ ]:
wrapped_retrieve.load("This is the best explanation for machine learning transformers:")
print(wrapped_retrieve.read("what is a transformer"))
print(wrapped_retrieve.read("who wrote the first paper on transformers"))

[Exa Tool] Autoprompt: Here's the best explanation for machine learning transformers:
A transformer is a type of semi-supervised machine learning model that has revolutionized natural language processing tasks in recent years. It is primarily used with text data and has replaced recurrent neural networks in many applications. Transformers work on sequence data and can take an input sequence and generate an output sequence one element at a time. They consist of an encoder, which operates on the input sequence, and a decoder, which operates on the target output sequence during training and predicts the next item in the sequence. Transformers have become the de facto standard for NLP tasks and have also been applied in other fields such as computer vision and music generation.


## Creating the Agent

We now are ready to create an Agent that can use Metaphors services to it's full potential. We will use our wrapped read and load tools, as well as the `get_date` utility for the following agent and test it out below:

In [ ]:
# Just pass the wrapped tools and the get_date utility
agent = OpenAIAgent.from_tools(
    [*wrapped_retrieve.to_tool_list(), date_tool],
    verbose=True,
)

In [ ]:
print(
    agent.chat(
        "Can you summarize everything published in the last month regarding news on"
        " superconductors"
    )
)

We asked the agent to retrieve documents related to superconductors from this month. It used the `get_date` tool to determine the current month, and then applied the filters in Metaphor based on publication date when calling `search`. It then loaded the documents using `retrieve_documents` and read them using `read_retrieve_documents`.

We can make another query to the vector store to read from it again, now that the articles are loaded: